In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import os

import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
frac = '0.5'

In [3]:
# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

#Importar canasta basica regional deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')

ppp_defl = pd.read_csv('./../data/info/ppp_defl.csv')

# Load radio ref. Merge regiones.
# Anything that is AGLOMERADO 33 should be region Gran Buenos Aires

radio_ref = pd.read_csv('./../data/info/radio_ref.csv').merge(pd.read_csv('./../data/info/prov_regs.csv'), how = 'left')

radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

# DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

In [4]:
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

DPTO_Region.Region = DPTO_Region.Region.map({'Gran Buenos Aires': 'gran_buenos_aires',
                                            'Pampeana': 'pampeana',
                                             'Noroeste': 'noroeste',
                                             'Noreste': 'noreste',
                                             'Patagónica': 'patagonia',
                                             'Cuyo': 'cuyo'})

## 1.  Resultados estaticos (se toma 1 año)

In [5]:
import glob
# path ='./data/RFReg_' # use your path
# path ='./../../encuestador-de-hogares/data/yr_samples/RFReg_' # use your path
path ='/media/miglesia/Elements/suite/yr_samples/RFReg_'
experiment_tag = 'hd_test'

allFiles = []
for year in [str(s) for s in range(2019, 2022)]:
    allFiles += glob.glob(path +str(frac)+ '*'+str(year)+'*_'+experiment_tag+'.csv')
    # Estos son los archivos que se usan para tener una figura estatica, corte donde no importa evol. temporal.

allFiles = sorted(allFiles)[-2:] # ultimo anio
allFiles

['/media/miglesia/Elements/suite/yr_samples/RFReg_0.5_2021-08-15_hd_test.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.5_2021-11-15_hd_test.csv']

In [6]:
# years = np.unique([int(f[-14:-10]) for f in  allFiles])
years = np.unique([int(f[-22:-18]) for f in  allFiles[-4:]]) # ultimo anio 
years = np.unique([int(f[-22:-18]) for f in  allFiles[-8:]]) # ultimos 2 anios
# f[-18:-14] # Depende de la convencion para nombrar el archivo

if len(years) == 1:
    yr_label = str(years[0])
else:
    yr_label = '-'.join([str(years.min()), str(years.max())])
    
yr_label

'2021'

In [7]:

df_parts = []
for quarter_Xy_file in sorted(allFiles)[-8:]:# ultimo anio
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO', 'PP07K'])
    df_Q['ANO4'] = int(quarter_Xy_file[-22:-18])
    q = quarter_Xy_file[-22:-12]; print(q)
    df_Q['Q'] = q
    df_parts += [df_Q]

df = pd.concat(df_parts)
del df_Q


2021-08-15
2021-11-15


In [8]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]

[('df', 77337846),
 ('radio_ref', 17219219),
 ('dpto_region', 120394),
 ('CB_ipc', 71584),
 ('DPTO_Region', 42613),
 ('ppp_defl', 8769),
 ('ad_eq', 5472)]

In [9]:

columnas_pesos = ['P47T']
df[columnas_pesos] = np.power(10, df[columnas_pesos]) - 1

# # df.PERSONA_REF_ID.nunique()
# 1756665 - 891932 - 884035 = -19302

# use means:
# ddf = dd.from_pandas(df, npartitions=50);
# with ProgressBar():
#     df = ddf.groupby(['PERSONA_REF_ID','Q']).mean().compute() 
## Hay una pequena cantidad de duplicados, gente que fue sampleada mas de una vez, y a las que el modelo les dio el mismo resultado en repetidas veces.
# El unico problemita que dan es que los hogares donde estan se agruparian en uno solo. 
# Quizas se corrige sacando el sampleo con replacement.
# Por ahora ignoro el detalle

# Editar columnas
df['P10'] = 2 - df['P10']
df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 

# df = df.astype(int)
df['P0910'] = df.P09.astype(str) + df.P10.astype(str)
# df['Grupo_Etario_3'] = pd.cut(df.P03, np.arange(-1, 80, 3))#.round(-1)
# df['Grupo_Etario_INDEC'] = pd.cut(df.P03, np.array([0, 14, 29, 64, 110]))#.round(-1)
# df['Grupo_Etario_q10'] = pd.cut(df.P03, np.array([-0.001, 5.0,  11.0, 17.0, 23.0, 29.0, 36.0, 44.0, 53.0, 65.0, 110.0]))#.round(-1)

df = df.reset_index()


In [10]:
# # Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]


[('df', 98963244),
 ('radio_ref', 17219219),
 ('dpto_region', 120394),
 ('CB_ipc', 71584),
 ('DPTO_Region', 42613),
 ('ppp_defl', 8769),
 ('ad_eq', 5472)]

In [11]:
df.groupby('Q').size()

Q
2021-08-15    183265
2021-11-15    183265
dtype: int64

In [12]:
df.shape

(366530, 20)

### Canasta y Pobreza

In [13]:
experiment_tag = 'hd_test'

In [14]:
df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)#.merge(ppp_defl[['Q', 'ppp_5usd_ARS_deflac']])

# df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV', 'ppp_5usd_ARS_deflac']].sum()
df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV']].sum()
# del df_cb # Ahorrar memoria

df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
# df_cb_hogares['Pobreza_5usd'] = df_cb_hogares['P47T'] < df_cb_hogares['ppp_5usd_ARS_deflac']
# pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia', 'Pobreza_5usd']].reset_index()
pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia']].reset_index()
pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

# df = df.sample(25000)
data = df.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
# del df; del pobreza_hogares # Ahorrar memoria

data = data.rename(columns = {'P47T': 'P47T_persona'})

# data = data
data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO']].drop_duplicates())

if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')
filename = './../data/Pobreza/pobreza_'+'_'.join([str(frac), yr_label, experiment_tag])+'.csv'
data.to_csv(filename, index = False)


In [15]:
filename 

'./../data/Pobreza/pobreza_0.5_2021_hd_test.csv'

In [16]:
data.groupby('RADIO_REF_ID').size().describe()

count     347.000000
mean     1056.282421
std       450.823628
min        32.000000
25%       782.000000
50%      1060.000000
75%      1309.000000
max      3114.000000
dtype: float64

## Sintetizar (Stats a nivel Radio)

In [17]:
# data[['Emp_blanco', 'Emp_negro', 'Monotrib', 'Otros_n.a.']].mean()

In [18]:
### Sintetizadores
import datetime as dt

## Base Personas
def sintetizar_P(data, grouper, frac = 0.1, mult = 1):
    df = data.copy()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()
    
#     # Funciones percentil
#     percentiles = [.10, .25, .75, .9, .2]
#     quantile_funcs = [(p, lambda x: x.quantile(p)) for p in percentiles]

    df = df.groupby(grouper + ['timestamp'])[['Total', 'Pobreza', 'Indigencia','P47T_persona', 'Emp_blanco', 'Emp_negro', 'Monotrib']].agg(
        {'Total': ['mean', 'sum'], 
         'Pobreza': ['mean', 'sum'], 
         'Indigencia': ['mean', 'sum'],
         'Emp_blanco': ['mean', 'sum'], 
         'Emp_negro': ['mean', 'sum'], 
         'Monotrib': ['mean', 'sum'], 
         'P47T_persona':['mean', q10, q25, 'median', q75, q90]})
    
    df[[('Total', 'sum')]] = (df[[('Total', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Pobreza', 'sum')]] = (df[[('Pobreza', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Indigencia', 'sum')]] = (df[[('Indigencia', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Emp_blanco', 'sum')]] = (df[[('Emp_blanco', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Emp_negro', 'sum')]] = (df[[('Emp_negro', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Monotrib', 'sum')]] = (df[[('Monotrib', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Total', 'mean')]] = df[[('Total', 'mean')]].round(4)
    df[[('Pobreza', 'mean')]] = df[[('Pobreza', 'mean')]].round(4)
    df[[('Indigencia', 'mean')]] = df[[('Pobreza', 'mean')]].round(4)
    df[[('Emp_blanco', 'mean')]] = df[[('Emp_blanco', 'mean')]].round(4)
    df[[('Emp_negro', 'mean')]] = df[[('Emp_negro', 'mean')]].round(4)
    df[[('Monotrib', 'mean')]] = df[[('Monotrib', 'mean')]].round(4)
    
    df['P47T_persona'] = df['P47T_persona'].round(-1).astype(int)
    
    dfB = df.T.set_index(np.repeat('Personas', df.shape[1]), append=True)
       
#     dfB = dfB.stack().stack().stack().reset_index()
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    dfB = dfB.stack(level = stacker_ix).reset_index()
    
    dfB = dfB.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    dfB['frac'] = frac
    return dfB

def sintetizar_H(data, grouper, frac = 0.01, mult = 1):
    df = data.copy()
    df = df.groupby('HOGAR_REF_ID').first()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()

#     # Funciones percentil
#     percentiles = [.10, .25, .75, .9, .2]
#     quantile_funcs = [(p, lambda x: x.quantile(p)) for p in percentiles]

    df = df.groupby(grouper + ['timestamp'])[['Total', 'Pobreza', 'Indigencia', 'P47T_hogar', 'CB_EQUIV', 'CBA', 
                                              'gap_indigencia', 'CBT', 'gap_pobreza']
                            ].agg({'Total': ['mean', 'sum'], 
                                   'Pobreza': ['mean', 'sum'], 
                                   'Indigencia': ['mean', 'sum'], 
                                   'P47T_hogar': ['mean', q10, q25, 'median', q75, q90],
                                   'CB_EQUIV': ['mean', 'median'],     
                                   'CBA': ['mean', 'median'], 
                                   'CBT': ['mean', 'median'], 
                                   'gap_indigencia': ['mean', 'median'], 
                                   'gap_pobreza': ['mean', 'median']})
        
    df[[('Total', 'sum')]] = (df[[('Total', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Pobreza', 'sum')]] = (df[[('Pobreza', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Indigencia', 'sum')]] = (df[[('Indigencia', 'sum')]]/frac/mult).round(1)#.astype(int)
    df[[('Total', 'mean')]] = df[[('Total', 'mean')]].round(4)
    df[[('Pobreza', 'mean')]] = df[[('Pobreza', 'mean')]].round(4)
    df[[('Indigencia', 'mean')]] = df[[('Indigencia', 'mean')]].round(4)

    df['CB_EQUIV'] = df['CB_EQUIV'].round(4)
    
    df['P47T_hogar'] = df['P47T_hogar'].round(-1).astype(int)
    df['CBA'] = df['CBA'].round(-1).astype(int)
    df['gap_indigencia'] = df['gap_indigencia'].round(-1).astype(int)
    df['CBT'] = df['CBT'].round(-1).astype(int)
    df['gap_pobreza'] = df['gap_pobreza'].round(-1).astype(int)
    
    dfC = df.T.set_index(np.repeat('Hogares', df.shape[1]), append=True)
    dfC.index.names = ['observable', 'sintetico', 'base']
    
#     FrozenList(['observable', 'sintetico', 'base', 'timestamp', 'AGLOMERADO', 'Region', 'Q'])
    
# #     dfC = dfC.stack().stack().stack().reset_index()
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    dfC = dfC.stack(level = stacker_ix).reset_index()
#     # es como df.stack([-1, -2, -3, -4]) pero se adapta a groupers mas o menos largos
#     dfC = dfC.stack(stacker_ix).reset_index()

    dfC = dfC.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    dfC['frac'] = frac
    return dfC

# Percentile funcs
def q10(x):
    return x.quantile(0.1)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def q90(x):
    return x.quantile(0.9)

In [19]:
data.columns

Index(['index', 'RADIO_REF_ID', 'DPTO', 'HOGAR_REF_ID', 'H15', 'H16',
       'PERSONA_REF_ID', 'P02', 'P03', 'P09', 'P10', 'CONDACT', 'IX_TOT',
       'CAT_OCUP', 'CAT_INAC', 'PP07K', 'P47T_persona', 'ANO4', 'Q', 'P0910',
       'P47T_hogar', 'CBA', 'CBT', 'CB_EQUIV', 'Pobreza', 'Indigencia',
       'gap_pobreza', 'gap_indigencia', 'IDFRAC', 'PROV', 'NOMPROV',
       'AGLOMERADO'],
      dtype='object')

In [20]:
## INDICE DE PRECIOS - CONVERSION A VALOR CONSTANTE

cpi_M = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi_M.index = pd.to_datetime(cpi_M.index)
  
# Getting current date and time
today = dt.datetime.today()
ref = '2016-01'

cpi_actual = cpi_M.loc[today.strftime("%Y-%m")]['index'][0]
cpi_ref = cpi_M.loc[ref]['index'][0]

# df_ARShoy = df*(cpi_actual/cpi_ref)

cpi_Q = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_Q.csv', index_col=0)
cpi_Q.index = pd.to_datetime(cpi_Q.index)
cpi_2021oct = cpi_M.loc['2021-10']['index'][0]
cpi_2021oct/cpi_ref

7.039412907526196

In [21]:
data = pd.read_csv('./../data/Pobreza/pobreza_0.5_2021_hd_test.csv')

data.P47T_persona = data.P47T_persona*(cpi_2021oct/cpi_ref)

data['Emp_blanco'] = (data.PP07K == 1)
data['Emp_negro'] = (data.PP07K != 1) & (data.CAT_OCUP == 3)
data['Monotrib'] = data.CAT_OCUP == 2
data['Otros_n.a.'] = ~data.CAT_OCUP.isin([2, 3])

In [22]:
grouper = ['PROV', 'DPTO', 'RADIO_REF_ID']
frac = .5
df_ = sintetizar_P(data, grouper, frac, mult = 2)

In [23]:
df_.nunique()

observable         7
sintetico          7
base               1
PROV               1
timestamp          1
RADIO_REF_ID     347
DPTO               2
valor           2723
frac               1
dtype: int64

In [27]:
df_.groupby(['observable', 'sintetico']).size()

observable    sintetico
Emp_blanco    mean         347
              sum          347
Emp_negro     mean         347
              sum          347
Indigencia    mean         347
              sum          347
Monotrib      mean         347
              sum          347
P47T_persona  mean         347
              median       347
              q10          347
              q25          347
              q75          347
              q90          347
Pobreza       mean         347
              sum          347
Total         mean         347
              sum          347
dtype: int64

In [25]:
df_.to_csv('./../data/results/sintetico_v1test.csv', index = False)